In [3]:
import numpy as np
import pyglet

In [2]:
class ArmEnv(object):
    viewer = None
    dt = 0.1    # refresh rate
    action_bound = [-1, 1]
    goal = {'x': 100., 'y': 100., 'l': 25}
    state_dim = 2
    action_dim = 2

    def __init__(self):
        self.arm_info = np.zeros(
            2, dtype=[('l', np.float32), ('r', np.float32)])
        self.arm_info['l'] = 100
        self.arm_info['r'] = np.pi/6

    def step(self, action):
        done = False
        r = 0.
        action = np.clip(action, *self.action_bound)
        self.arm_info['r'] += action * self.dt
        self.arm_info['r'] %= np.pi * 2    # normalize

        # state
        s = self.arm_info['r']

        (a1l, a2l) = self.arm_info['l']  # radius, arm length
        (a1r, a2r) = self.arm_info['r']  # radian, angle
        a1xy = np.array([200., 200.])    # a1 start (x0, y0)
        a1xy_ = np.array([np.cos(a1r), np.sin(a1r)]) * a1l + a1xy  # a1 end and a2 start (x1, y1)
        finger = np.array([np.cos(a1r + a2r), np.sin(a1r + a2r)]) * a2l + a1xy_  # a2 end (x2, y2)

        # done and reward
        if (self.goal['x'] - self.goal['l']/2 < finger[0] < self.goal['x'] + self.goal['l']/2
        ) and (self.goal['y'] - self.goal['l']/2 < finger[1] < self.goal['y'] + self.goal['l']/2):
                done = True
                r = 1. # Sparse reward
        return s, r, done
    
    def reset(self):
        self.arm_info['r'] = 2 * np.pi * np.random.rand(2)
        return self.arm_info['r']

    def render(self):
        if self.viewer is None:
            self.viewer = Viewer(self.arm_info, self.goal)
        self.viewer.render()

    def sample_action(self):
        return np.random.rand(2)-0.5    # two radians

In [3]:
class Viewer(pyglet.window.Window):
    bar_thc = 5
    w0 = 25   # Object width
    wp = 5.0  # 
    fw = 1.8  # Finger width
    
    d2 = 5.0  # distance from x-axis to right bottom of the object 
    t2 = np.pi / 2
    t1 = np.pi / 2
    d1 = 6.2 # distance from x-axis to left bottom of the object
    def __init__(self, arm_info, goal):
        # vsync=False to not use the monitor FPS, we can speed up training
        super(Viewer, self).__init__(width=400, height=400, resizable=False, caption='Arm', vsync=False)
        pyglet.gl.glClearColor(1, 1, 1, 1)
        self.arm_info = arm_info
        self.center_coord = np.array([200, 200])
        
        self.obj_pos = translateLeft(self.t2, self.d2)
        self.batch = pyglet.graphics.Batch()    # display whole batch at once
        """
        self.goal = self.batch.add(
            4, pyglet.gl.GL_QUADS, None,    # 4 corners
            ('v2f', [goal['x'] - goal['l'] / 2, goal['y'] - goal['l'] / 2,                # location
                     goal['x'] - goal['l'] / 2, goal['y'] + goal['l'] / 2,
                     goal['x'] + goal['l'] / 2, goal['y'] + goal['l'] / 2,
                     goal['x'] + goal['l'] / 2, goal['y'] - goal['l'] / 2]),
            ('c3B', (86, 109, 249) * 4))    # color
        """
        self.goal = self.batch.add(
            4, pyglet.gl.GL_QUADS, None,    # 4 corners
            ('v2f', [self.obj_pos[0][0], self.obj_pos[0][1]         # location
                     self.obj_pos[1][0], self.obj_pos[1][1],
                     self.obj_pos[2][0], self.obj_pos[2][1],
                     self.obj_pos[3][0], self.obj_pos[3][1]),
            ('c3B', (86, 109, 249) * 4))    # color
        
        self.arm1 = self.batch.add(
            4, pyglet.gl.GL_QUADS, None,
            ('v2f', [250, 250,                # location
                     250, 300,
                     260, 300,
                     260, 250]),
            ('c3B', (249, 86, 86) * 4,))    # color
        self.arm2 = self.batch.add(
            4, pyglet.gl.GL_QUADS, None,
            ('v2f', [100, 150,              # location
                     100, 160,
                     200, 160,
                     200, 150]), ('c3B', (249, 86, 86) * 4,))
        
    def translateLeft(t2, d2):
        
        # Transformation Matrix
        x_square = self.wp + (d2 + w0 / 2.) * np.cos(np.float64(t2)) - (self.fw + self.w0 / 2.) * np.sin(np.float64(t2))
        y_square = (d2 + w0 / 2.) * np.sin(np.float64(t2)) + (self.fw + self.w0 / 2.) * np.cos(np.float64(t2))

        pts = np.array([[-self.w0 / 2., -self.w0 / 2., self.w0 / 2., self.w0 / 2.], [-self.w0 / 2., self.w0 / 2., self.w0 / 2., -self.w0 / 2.], [1, 1, 1, 1]])
        R = np.array([[np.cos(t2), -np.sin(t2), x_square], [np.sin(t2), np.cos(t2), y_square], [0, 0, 1]])

        # Points after transformation
        pts_new = np.dot(R, pts)
        
        # Plotting the Object
        pts = np.transpose([[pts_new[0, :]], [pts_new[1, :]]])
        pts = pts.reshape((4, 2))
        return pts

    def render(self):
        self._update_arm()
        self.switch_to()
        self.dispatch_events()
        self.dispatch_event('on_draw')
        self.flip()

    def on_draw(self):
        self.clear()
        self.batch.draw()

    def _update_arm(self):
        (a1l, a2l) = self.arm_info['l']     # radius, arm length
        (a1r, a2r) = self.arm_info['r']     # radian, angle
        a1xy = self.center_coord            # a1 start (x0, y0)
        a1xy_ = np.array([np.cos(a1r), np.sin(a1r)]) * a1l + a1xy   # a1 end and a2 start (x1, y1)
        a2xy_ = np.array([np.cos(a1r+a2r), np.sin(a1r+a2r)]) * a2l + a1xy_  # a2 end (x2, y2)

        a1tr, a2tr = np.pi / 2 - self.arm_info['r'][0], np.pi / 2 - self.arm_info['r'].sum()
        xy01 = a1xy + np.array([-np.cos(a1tr), np.sin(a1tr)]) * self.bar_thc
        xy02 = a1xy + np.array([np.cos(a1tr), -np.sin(a1tr)]) * self.bar_thc
        xy11 = a1xy_ + np.array([np.cos(a1tr), -np.sin(a1tr)]) * self.bar_thc
        xy12 = a1xy_ + np.array([-np.cos(a1tr), np.sin(a1tr)]) * self.bar_thc

        xy11_ = a1xy_ + np.array([np.cos(a2tr), -np.sin(a2tr)]) * self.bar_thc
        xy12_ = a1xy_ + np.array([-np.cos(a2tr), np.sin(a2tr)]) * self.bar_thc
        xy21 = a2xy_ + np.array([-np.cos(a2tr), np.sin(a2tr)]) * self.bar_thc
        xy22 = a2xy_ + np.array([np.cos(a2tr), -np.sin(a2tr)]) * self.bar_thc

        self.arm1.vertices = np.concatenate((xy01, xy02, xy11, xy12))
        self.arm2.vertices = np.concatenate((xy11_, xy12_, xy21, xy22))


SyntaxError: invalid syntax (<ipython-input-3-e44e4d50dbe3>, line 32)

In [4]:
if __name__ == '__main__':
    env = ArmEnv()
    while True:
        env.render()
        #env.step(env.sample_action())

NameError: name 'Viewer' is not defined

In [7]:
for i in range(5):
    print(np.random.rand(1)-0.5)

[0.03341629]
[0.16985617]
[-0.36436752]
[-0.06954137]
[0.20358685]


In [6]:
action = np.random.rand(2)
print("action : ", action)

action = action - 0.5
print("action : ", action)

action :  [0.63924866 0.39260701]
action :  [ 0.13924866 -0.10739299]


In [3]:
action_bound = [-1, 1]
dt = 0.1


In [8]:
arm_info = np.zeros(3, dtype=[('l', np.float32), ('r', np.float32)])
arm_info

array([(0., 0.), (0., 0.), (0., 0.)], dtype=[('l', '<f4'), ('r', '<f4')])

In [14]:
arm_info['l'] = 100
arm_info['r'] = np.pi/6
arm_info

array([(100., 0.5235988), (100., 0.5235988), (100., 0.5235988)],
      dtype=[('l', '<f4'), ('r', '<f4')])

In [21]:
action_ = np.random.rand(1)
action_
print(action_)
action_bound = [-1, 1]
action_ = action_-0.5
print("action : ", action)

[0.55198386]
action :  [-0.05706867]


In [22]:
action_ = np.clip(action_, *action_bound)
action_

array([0.05198386])

In [77]:
ff_info = np.zeros(2, dtype=[('d', np.float32), ('t', np.float32), ('a', np.int)] )
print(ff_info)
ff_info['t'] = np.pi/2
ff_info['d'] = 62
print(ff_info)
print(ff_info['d'][0])

[(0., 0., 0) (0., 0., 0)]
[(62., 1.5707964, 0) (62., 1.5707964, 0)]
62.0


In [5]:
class ff_env(object):

    
    d2 = 50  # distance from x-axis to right bottom of the object 
    t2 = np.pi / 2 
    t1 = np.pi / 2
    d1 = 62 # distance from x-axis to left bottom of the object
    
    def __init__(self):
        self.action_bound = [-1, 1]
        self.ff_info = np.zeros(2, dtype=[('f', np.float32), ('t', np.float32)] )
        self.ff_info['t'] = np.pi/2
        self.ff_info['d'] = 0
        
        self.w0 = 25   # Object width
        self.wp = 50  # 
        self.fw = 18  # Finger width
    
    def reset(self):
        self.ff_info['t'] = np.pi/2
        
    def sample_action(self):
        return np.random.rand(1)-0.5
    
    def translateLeft_obj(self,t2, d2):
        
        # Transformation Matrix
        x_square = self.wp + (d2 + self.w0 / 2.) * np.cos(np.float64(t2)) - (self.fw + self.w0 / 2.) * np.sin(np.float64(t2)) 
        y_square = (d2 + self.w0 / 2.) * np.sin(np.float64(t2)) + (self.fw + self.w0 / 2.) * np.cos(np.float64(t2))

        pts = np.array([[-self.w0 / 2., -self.w0 / 2., self.w0 / 2., self.w0 / 2.], [-self.w0 / 2., self.w0 / 2., self.w0 / 2., -self.w0 / 2.], [1, 1, 1, 1]])
        R = np.array([[np.cos(t2), -np.sin(t2), x_square], [np.sin(t2), np.cos(t2), y_square], [0, 0, 1]])

        # Points after transformation
        pts_new = np.dot(R, pts)
        
        # Plotting the Object
        pts = np.transpose([[pts_new[0, :]], [pts_new[1, :]]])
        pts = pts.reshape((4, 2))
        """
        print(pts[1][0],pts[1][1], '\n',        # location
              pts[0][0],pts[0][1], '\n',
              pts[3][0],pts[3][1], '\n',
              pts[2][0],pts[2][1]) 
        """
        return pts*2.5

    def translateLeft_fingers(self,t2, d2):

        # Calculate theta2, d2
        d2v = np.array([d2 * np.cos(np.float64(t2)), d2 * np.sin(np.float64(t2))])
        w0v = np.array([self.w0 * np.sin(np.float64(t2)), -self.w0 * np.cos(np.float64(t2))])
        wpv = np.array([self.wp, 0.])
        f1v = np.array([self.fw * np.sin(np.float64(t2)), -self.fw * np.cos(np.float64(t2))])
        av = d2v - f1v - w0v + wpv
        # Calculated Values of theta1, d1
        d1 = np.sqrt(float((av * av).sum() - self.fw * self.fw))
        t1 = np.arctan2(float(av[1]), float(av[0])) + np.arctan2(self.fw, d1)

        l_fw_pts = np.array([[0., 0., self.fw, self.fw], [40, 130, 130, 40], [1.0, 1.0, 1.0, 1.0]])
        r_fw_pts = np.array([[0., 0., -self.fw, -self.fw], [40, 130, 130, 40], [1.0, 1.0, 1.0, 1.0]])
        # Transformation matrices for the finger width
        R_fw1 = [[np.cos(t1 - np.pi / 2.0), -np.sin(t1 - np.pi / 2), 0.0], [np.sin(t1 - np.pi / 2), np.cos(t1 - np.pi / 2), 0.0], [0.0, 0.0, 1.0]]
        R_fw2 = [[np.cos(t2 - np.pi / 2), -np.sin(t2 - np.pi / 2), self.wp], [np.sin(t2 - np.pi / 2), np.cos(t2 - np.pi / 2), 0.0], [0.0, 0.0, 1.0]]

        # finger Coordinates 1-> Left, 2-> Right
        pts_fw1 = np.dot(R_fw1, l_fw_pts)
        pts_fw2 = np.dot(R_fw2, r_fw_pts)

        # Plotting the fingers
        fw_1 = np.transpose([[pts_fw1[0, :]], [pts_fw1[1, :]]]).reshape((4, 2))
        fw_2 = np.transpose([[pts_fw2[0, :]], [pts_fw2[1, :]]]).reshape((4, 2))
        #print("fw_1 :", fw_1)
        #print("fw_2 :", fw_2)
        return fw_1*2.5, fw_2*2.5
    

In [6]:
ffenv = ff_env()

d2 = 50  # distance from x-axis to right bottom of the object 
t2 = np.pi / 2 
t1 = np.pi / 2
d1 = 62 # distance from x-axis to left bottom of the object

dt = 0.1


print(ffenv.ff_info)

obj_pos = ffenv.translateLeft_obj(t2, d2)
print("obj_pos : ", obj_pos)
finger_l, finger_r = ffenv.translateLeft_fingers(t2, d2)
print("finger_l : ", finger_l)
print("finger_r : ", finger_r)

"""
print('---------------')
for i in range(10):
    action = ffenv.sample_action()
    print(action)
    action = np.clip(action, *action_bound)
    print("clipped action : ", action)
    print("t2 initial : ", t2)
    t2 += action * dt
    print("t2 changed : ", t2)
    obj_pos = ffenv.translateLeft_obj(t2, d2)
    print("obj_pos : ", obj_pos)
    finger_l, finger_r = ffenv.translateLeft_fingers(t2, d2)
    print("finger_l : ", finger_l)
    print("finger_r : ", finger_r)
"""

[(0., 1.5707964) (0., 1.5707964)]
obj_pos :  [[ 80.  125. ]
 [ 17.5 125. ]
 [ 17.5 187.5]
 [ 80.  187.5]]
finger_l :  [[-22.35428248  97.46941087]
 [-72.65141806 316.77558532]
 [-28.79018316 326.83501243]
 [ 21.50695241 107.52883798]]
finger_r :  [[125. 100.]
 [125. 325.]
 [ 80. 325.]
 [ 80. 100.]]


'\nprint(\'---------------\')\nfor i in range(10):\n    action = ffenv.sample_action()\n    print(action)\n    action = np.clip(action, *action_bound)\n    print("clipped action : ", action)\n    print("t2 initial : ", t2)\n    t2 += action * dt\n    print("t2 changed : ", t2)\n    obj_pos = ffenv.translateLeft_obj(t2, d2)\n    print("obj_pos : ", obj_pos)\n    finger_l, finger_r = ffenv.translateLeft_fingers(t2, d2)\n    print("finger_l : ", finger_l)\n    print("finger_r : ", finger_r)\n'

In [7]:
obj_pos[1][0], obj_pos[1][1], obj_pos[0][0]

(17.500000000000018, 125.0, 80.00000000000001)

In [16]:
w0 = 25   # Object width
wp = 50  # 
fw = 18  # Finger width

d2 = 50  # distance from x-axis to right bottom of the object 
t2 = np.pi / 2 
t1 = np.pi / 2
d1 = 62 # distance from x-axis to left bottom of the object
def translateLeft_obj(t2, d2):

    # Transformation Matrix
    x_square = wp + (d2 + w0 / 2.) * np.cos(np.float64(t2)) - (fw + w0 / 2.) * np.sin(np.float64(t2)) 
    y_square = (d2 + w0 / 2.) * np.sin(np.float64(t2)) + (fw + w0 / 2.) * np.cos(np.float64(t2))

    pts = np.array([[-w0 / 2., -w0 / 2., w0 / 2., w0 / 2.], [-w0 / 2., w0 / 2., w0 / 2., -w0 / 2.], [1, 1, 1, 1]])
    R = np.array([[np.cos(t2), -np.sin(t2), x_square], [np.sin(t2), np.cos(t2), y_square], [0, 0, 1]])

    # Points after transformation
    pts_new = np.dot(R, pts)

    # Plotting the Object
    pts = np.transpose([[pts_new[0, :]], [pts_new[1, :]]])
    pts = pts.reshape((4, 2))
    """    print(pts[1][0],pts[1][1], '\n',        # location
          pts[0][0],pts[0][1], '\n',
          pts[3][0],pts[3][1], '\n',
          pts[2][0],pts[2][1]) 
    """
    return pts*2.5

In [19]:
obj_pos_ = translateLeft_obj(np.pi/2, d2 =50)
print(obj_pos_)

[[ 80.  125. ]
 [ 17.5 125. ]
 [ 17.5 187.5]
 [ 80.  187.5]]


In [25]:
np.hstack([obj_pos_[1][0], obj_pos_[1][1]]) 

array([ 17.5, 125. ])

In [20]:
object = np.concatenate((obj_pos_[1][0], obj_pos_[1][1],         
                         obj_pos_[0][0], obj_pos_[0][1],
                         obj_pos_[3][0], obj_pos_[3][1],
                         obj_pos_[2][0], obj_pos_[2][1]))

ValueError: zero-dimensional arrays cannot be concatenated

In [26]:
np.pi

3.141592653589793

Limitations for the finger rotation 

40 deg : 0.698132 rad

140 deg : 2.443 rad

# Dual Action 

1. Complete generalising the environemnt for both the actions

Select a random action between 0 and 1 (delta_theta_left and delta_theta_right)

Create a common function for getting object positions, finger_right positions and finger_left positions based on action is delta theta_1 (or) delta tehta_2.  


In [3]:
import random
import numpy as np
list_ = [0, 1]
for i in range(5):
    list_i = random.choice(list_) 
    print(list_i) # Use this for randomly selecting the theta_l or tehta_r

0
0
1
0
0


In [4]:
for i in range(10):
    action = np.random.rand(1) - 0.5 # Use this for selecting the magnitude of change in tehta_l or theta_r
    print(action) 

[-0.35919346]
[-0.11897727]
[-0.1897613]
[0.02698582]
[0.43727288]
[-0.44523617]
[-0.44626876]
[0.28333861]
[-0.02326691]
[0.09613885]


In [5]:
list_ = [0, 1]
list_i = random.choice(list_) 
action_i = np.random.rand(1) - 0.5
action = np.random.rand(2)
if (list_i == 0):
    action[0] = action_i
    action[1] = 0
else:
    action[0] = 0
    action[1] = action_i

print(action)

[0.20258222 0.        ]


In [16]:
# Action[0]
if (action[0] != 0 and action[1] == 0):
    print("Aciton is sliding left")

Aciton is sliding left


In [14]:
def take_action():
    list_ = [0, 1]
    list_i = random.choice(list_) 
    action_i = np.random.rand(1) - 0.5
    action = np.random.rand(2)
    if (list_i == 0):
        action[0] = action_i
        action[1] = 0
    else:
        action[0] = 0
        action[1] = action_i

    print(action)
for i in range(10):
    take_action()
    

[ 0.         -0.21584845]
[0.         0.25775574]
[-0.17061405  0.        ]
[0.05357847 0.        ]
[-0.45357418  0.        ]
[0.37080079 0.        ]
[ 0.         -0.32791881]
[ 0.         -0.39942426]
[ 0.         -0.11572681]
[0.         0.06745553]


In [6]:
if (action[0] == 0):
    print('left')
elif (action[1] == 0):
    print('right')

right


In [7]:
action

array([0.20258222, 0.        ])

In [8]:
action[1] == 0

True

# Fixing calc_right_config

In [1]:
import numpy as np
import pyglet
import random
import time
from pyglet.window import key
from math import radians, degrees

In [2]:
tl = degrees(100)
dl = 100
w0 = 25
wp = 50
fw = 18

In [3]:
d1v = np.array([dl * np.cos(tl), dl * np.sin(tl)])
print(d1v)

[ 77.31211771 -63.42583429]


In [4]:
w0v = np.array([w0 * np.sin(tl), w0 * np.cos(tl)])
print(w0v)

[-15.85645857  19.32802943]


In [5]:
wpv = np.array([wp, 0.])
print(wpv)

[50.  0.]


In [6]:
f1v = np.array([fw * np.sin(tl), fw * np.cos(tl)])
print(f1v)

[-11.41665017  13.91618119]


In [7]:
av = d1v + w0v + f1v - wpv
print(av)

[  0.03900897 -30.18162367]


In [8]:
dr = np.sqrt(float((av * av).sum() - fw * fw))
print(dr)

24.22667804724324


In [11]:
np.arctan2(float(av[1]), float(av[0])) 

-1.5695038534225954

In [12]:
np.arctan2(float(av[1]), float(av[0])) - np.arctan2(fw, dr)

-2.208498671350577

In [9]:
tr = np.arctan2(float(av[1]), float(av[0])) - np.arctan2(fw, dr)
print(tr)

-2.208498671350577


In [10]:
print("av :", av)
print("fw :", fw)
print("dr :", dr)

av : [  0.03900897 -30.18162367]
fw : 18
dr : 24.22667804724324
